In [105]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
import copy

In [2]:
data_path = '/Users/peter/Downloads/compas-scores-two-years.csv'

In [18]:
df = pd.read_csv(data_path)

In [298]:
#helper function testing environment::
def helper_data_cleaning(x):
    #This code subsets the dataframe to relevant fields for the purpose of this analysis
    df = x[x['race'].isin(['African-American','Caucasian'])]
    idx = np.where((df['days_b_screening_arrest']<=30) & (df['days_b_screening_arrest']>=-30) &
                   (df['is_recid']!=-1) & (df['c_charge_degree']!="O") & (df['score_text']!="N/A") &
                   ((df['race']=="African-American") |(df['race']=="Caucasian")))
    df = df.iloc[idx]
    df=df.filter(items=['raw_data', 'age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
                    'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out'])
    
    return df

def encoding_data(x):
    
    x['race'] = x['race'].apply(lambda x: 1 if x == "Caucasian" else 0)
    #print(x['race'])
    x['sex'] = x['sex'].apply(lambda x: 1 if x == "Female" else 0)
    
    df1 = x
    df1['length_of_stay']=df1['c_jail_out'].apply(pd.to_datetime) - df1['c_jail_in'].apply(pd.to_datetime)
    df1['length_of_stay']=df1['length_of_stay'].dt.days
    df1['length_of_stay'] = df1.length_of_stay.apply(lambda x: 2  if x >100 else (1 if x >10 else 0))
    df1['priors_count'] = df1['priors_count'].apply(lambda x: 0 if x == 0 else x)
    df1['priors_count'] = df1['priors_count'].apply(lambda x: 1 if (1 <= x <= 3) else x)
    df1['priors_count'] = df1['priors_count'].apply(lambda x: 2 if x > 3 else x)
    # Label Encoding 
    #df1['race'] = df1['race'].apply(lambda x: 1 if x == 'Caucasian' else 0, axis = 0)
    #df1['race'] = df1['race_2']
    #df1.loc[df1.race=='Caucasian']=1
    #df1.loc[df1.race=='African-American']=0

    categorical_variables = ['c_charge_degree','sex','age_cat','score_text','length_of_stay','race']
    for var in categorical_variables:
        df1[var] = df1[var].astype('category').cat.codes
    final_vars = ['sex','age_cat','race','priors_count','c_charge_degree','length_of_stay']
    df1 = df1[final_vars]
    y = x['two_year_recid']
    return y,df1


In [664]:
#cleaning/prepping data
df_1 = helper_data_cleaning(df)
y, df_1 = encoding_data(df_1)

In [645]:
def unique_information_coef(x, y):
    import itertools
    nrow = x.shape[0]
    x_shap = x.shape[1]
    z = np.concatenate((x,y), axis=1)
    
    #unique combinations of possible encodings
    unique_combo = []
    for r in z.T:
        unique_combo.append(np.unique(r).tolist())
    
    cartesian = list(itertools.product(*unique_combo))
    
    running_p = 0
    
    
    for possible in cartesian:
        #print(ff," ", possible)
        mutual_ct, r_1_ct, r_2_ct = 0,0,0
        mutual_ct = np.sum(np.all(possible == z, axis=1))
        r_1_ct = np.sum(np.all(possible[:x_shap] == z[:,:x_shap], axis=1))
        r_2_ct = np.sum(np.all(possible[x_shap:] == z[:,x_shap:], axis=1))
        
        #for row in z:
            #checking if joint
         #   if possible == row:
          #      mutual_ct = mutual_ct+1 
            #checking x
           # if possible[:x_shap]== z[:x_shap]:
            #    r_1_ct = r_1_ct +1
            #checking y
            #if possible[x_shap:]== z[x_shap:]:
             #   r_2_ct = r_2_ct +1
            
        
        #saving computation in the event there's a zero result
        if (mutual_ct == 0 or r_1_ct == 0 or r_2_ct == 0):
            intermed = 0
        else:
            mutual_p = mutual_ct/nrow
            pr1 = r_1_ct/nrow
            pr2 = r_2_ct/nrow
            intermed = mutual_p * np.log(mutual_p / pr1) / pr1
        running_p += abs(intermed)
        
        
    return running_p

def conditional_info_coef(x,y,c):
    import itertools
    nrow = x.shape[0]
    x_shap = x.shape[1]
    y_shap = y.shape[1]
    #print(x.shape,y.shape,c.shape)
    z = np.concatenate((y,x,c), axis=1)
    
    #unique combinations of possible encodings
    unique_combo = []
    for r in z.T:
        unique_combo.append(np.unique(r).tolist())
    
    cartesian = list(itertools.product(*unique_combo))
    
    running_p = 0
    
    
    for possible in cartesian:
        
        #print(ff," ", possible)
        mutual_ct, r_1_ct, r_2_ct, r_cond = 0,0,0,0
        mutual_ct = np.sum(np.all(possible == z, axis=1))
        # r_1_ct = np.sum(np.all(possible[:1] == z[:,:1], axis=1))
        # r_2_ct = np.sum(np.all(possible[1:-x_shap] == z[:,1:-x_shap], axis=1))
        r_1_ct = np.sum(np.all(possible[:y_shap] == z[:,:y_shap], axis=1))
        r_2_ct = np.sum(np.all(possible[y_shap:-x_shap] == z[:,y_shap:-x_shap], axis=1))
        cond_ct_num = np.sum(np.where((possible[:y_shap] == z[:,:y_shap]).all(axis=1) & (possible[-x_shap:] == z[:,-x_shap:]).all(axis=1)))
        cond_ct_den = np.sum(np.where((possible[-x_shap:] == z[:,-x_shap:]).all(axis=1)))
        if cond_ct_den == 0:
            r_cond = 0
        else:
            r_cond = cond_ct_num/cond_ct_den
        #for row in z:
            #checking if joint
         #   if possible == row:
          #      mutual_ct = mutual_ct+1 
            #checking x
           # if possible[:x_shap]== z[:x_shap]:
            #    r_1_ct = r_1_ct +1
            #checking y
            #if possible[x_shap:]== z[x_shap:]:
             #   r_2_ct = r_2_ct +1
            
        
        #saving computation in the event there's a zero result
        if (mutual_ct == 0 or r_1_ct == 0 or r_2_ct == 0 or r_cond ==0):
            intermed = 0
        else:
            mutual_p = mutual_ct/nrow
            pr1 = r_1_ct/nrow
            pr2 = r_2_ct/nrow
            intermed = mutual_p * np.log(mutual_p / pr2) / r_cond
    
        running_p += abs(intermed)
        
        
    return running_p

def powerset(seq):
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item


In [660]:
#MY SHAPELY DISCRIMINATION
def shapelydiscrimination(x,y):
    
    features = list(x.columns)
    features = [ele for ele in features if ele != 'race'] #removing race from list of features over which we iterate
    nfeat = len(features)
    
    shapely_coeff = []
    
    #identifying a feature sequentially to exclude from infoset calc
    for i in range(nfeat):
        
        features_2 = features.copy()
        rem = features_2.pop(i) #removing a feature i programmatically
        pow_set = [elm for elm in powerset(features_2)] #generating powerset list of lists
        
        shapely_inter = 0 #for summing the shapelys
        #now, iterating over the subsets
        for subset in pow_set:
            s_len = len(subset)
            f_len = nfeat - 1 - len(subset)
            
            m_coefficient = math.factorial(s_len) * math.factorial(f_len) / math.factorial(nfeat) #coeff
            
            #inclusive discrimination metric
            inc_sub = subset.copy()
            inc_sub.append(rem)
            
            x_1 = np.array(x[inc_sub])
            #print("x:",)
            protected_attribute = np.array(x['race']).reshape(-1,1)
            #print(protected_attribute.shape)
            y_1 = np.array(y).reshape(-1,1)
            
            info_in_ = np.concatenate((x_1,protected_attribute), axis = 1)
            
            incl_a = unique_information_coef(info_in_,y_1)
            incl_b = unique_information_coef(x_1,protected_attribute)
            incl_c = conditional_info_coef(x_1,protected_attribute,y_1)
            
            incl = incl_a * incl_b * incl_c
            #call info right here
            
            #exclusive discrimination metric
            x_2 = np.array(x[subset])
            info_ex_ = np.concatenate((x_2,protected_attribute), axis = 1)
            excl_a = unique_information_coef(info_ex_,y_1)
            excl_b = unique_information_coef(x_2,protected_attribute)
            excl_c = conditional_info_coef(x_2,protected_attribute,y_1)
            excl = excl_a * excl_b * excl_c
            
            #call info right here
            marginal_discrimination = incl-excl
            #marginal = disc_incl - disc_excl
            #shapley_coeff[i] = shapley_coeff[i] + coef * marginal
            shapely_inter = shapely_inter + m_coefficient * marginal_discrimination
        shapely_coeff.append(shapely_inter)
       
    return shapely_coeff

def shapelyaccuracy(x,y):
    features = list(x.columns)
    features = [ele for ele in features if ele != 'race'] #removing race from list of features over which we iterate
    nfeat = len(features)
    
    shapely_coeff = []
    
    #identifying a feature sequentially to exclude from infoset calc
    for i in range(nfeat):
        
        features_2 = features.copy()
        rem = features_2.pop(i) #removing a feature i programmatically
        pow_set = [elm for elm in powerset(features_2)] #generating powerset list of lists
        
        shapely_inter = 0 #for summing the shapelys
        #now, iterating over the subsets
        for subset in pow_set:
            s_len = len(subset)
            f_len = nfeat - 1 - len(subset)
            
            m_coefficient = math.factorial(s_len) * math.factorial(f_len) / math.factorial(nfeat) #coeff
            
            #inclusive accuracy metric
            inc_sub = subset.copy()
            inc_sub.append(rem)
            inc_sub_diff = list(set(features)-set(inc_sub))
            x_1c = np.array(x[inc_sub_diff])
            x_1 = np.array(x[inc_sub])
            #print("x:",)
            protected_attribute = np.array(x['race']).reshape(-1,1)
            #print(protected_attribute.shape)
            y_1 = np.array(y).reshape(-1,1)
            
            c = np.concatenate((x_1c,protected_attribute), axis = 1)
            
            #print( "out2: ",get_conditional_info_coef(y_1,x_1,c))
            accuracy_inclusive = conditional_info_coef(x_1,y_1,c)
            #call info right here
            
            #exclusive accuracy metric
            x_2 = np.array(x[subset])
            excl_sub = features_2
            excl_sub_diff = list(set(features_2)-set(subset))
            x_2c = np.array(x[excl_sub_diff])
            d = np.concatenate((x_2c,protected_attribute), axis = 1)
            accuracy_exclusive = conditional_info_coef(x_2,y_1,d)
            
            #marginal accuracy by including
            marginal_acc = accuracy_inclusive - accuracy_exclusive
            shapely_inter = shapely_inter + m_coefficient * marginal_acc
            
        if shapely_inter> 0:
            shapely_coeff.append(shapely_inter)
    return shapely_coeff
    

In [661]:
accuracy = shapelyaccuracy(df_1,y)

In [658]:
discrimination = shapelydiscrimination(df_1,y)

In [657]:
features = list(df_1.columns)
features = [ele for ele in features if ele != 'race']

Index(['sex', 'age_cat', 'race', 'priors_count', 'c_charge_degree',
       'length_of_stay'],
      dtype='object')

In [665]:
acc_disc_pretty = pd.DataFrame(list(zip(features,accuracy,discrimination)))

In [667]:
acc_disc_pretty.columns = ['variable','accuracy','discrimination']

In [668]:
acc_disc_pretty

,variable,accuracy,discrimination
0,sex,0.664604,5933.746182
1,age_cat,0.974554,8871.607730
2,priors_count,0.967169,9024.280430
3,c_charge_degree,0.780089,7155.980280
4,length_of_stay,0.732485,8352.821957
